### Include Extracted Text to the Huggingface Embeddings

In [1]:
import sys
import os

# Add the root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [1]:
from caching import load_full_hf_cache
import pandas as pd
import dask.array as da
import dask
import requests
import numpy as np
import stackview
from io import BytesIO
from pdf2image import convert_from_bytes
import dask.array as da
import numpy as np
import umap.umap_ as umap


repo_name = "ScaDS-AI/SlightInsight_Cache"
df = load_full_hf_cache(repo_name=repo_name)

In [2]:
df.head()

,key,zenodo_record_id,zenodo_filename,page_number,text,visual,mixed
0,record10008464_pdf1_slide1,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,1,"[0.8054575, -0.4204579, 0.111390926, 0.2636367...","[0.22024887800216675, 0.6564452648162842, 0.02...","[0.22330284, -0.5643485, 0.32713842, -0.074667..."
1,record10008464_pdf1_slide2,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,2,"[1.1522328, 0.02467385, 0.23145455, 0.17258961...","[-0.3846272826194763, -0.01668522134423256, -0...","[0.34757608, -0.6562839, 0.5335755, -0.1906935..."
2,record10008464_pdf1_slide3,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,3,"[1.0057557, 0.18344171, 0.03796136, 0.23549518...","[-0.32376205921173096, 0.06897055357694626, 0....","[0.06456853, -0.79703176, 0.871621, -0.8768049..."
3,record10008464_pdf1_slide4,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,4,"[1.2563236, 0.25261688, 0.0040982994, 0.202414...","[-0.03493745997548103, -0.11242333054542542, 0...","[0.3643795, -0.40593308, 0.97395766, -0.495041..."
4,record10008464_pdf1_slide5,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,5,"[0.8471789, -0.40933934, 0.85754687, -0.496454...","[-0.6693567037582397, -0.02434205450117588, -0...","[-0.22939722, -0.6752343, 0.57570195, 0.046955..."


In [4]:
import requests
import pdfplumber
from io import BytesIO
import os

pdf_folder = "zenodo_pdfs"
os.makedirs(pdf_folder, exist_ok=True)

# Zenodo API base URL
ZENODO_API_BASE = "https://zenodo.org/api/records/"

# Step 1: Select all unique Zenodo record IDs
unique_record_ids = df["zenodo_record_id"].unique()

# Step 1: Function to fetch Zenodo record files (PDFs)
def get_zenodo_pdfs(record_id):
    api_url = f"https://zenodo.org/api/records/{record_id}"
    response = requests.get(api_url)

    if response.status_code != 200:
        print(f"Failed to fetch Zenodo record {record_id}, skipping...")
        return []

    record_data = response.json()
    pdf_files = sorted(
        [file for file in record_data.get("files", []) if file["key"].lower().endswith(".pdf")],
        key=lambda x: x["key"]  # Sort by filename
    )
    return pdf_files

# Step 2: Function to download a PDF file from Zenodo
def download_pdf(pdf_url):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        return BytesIO(response.content)
    else:
        print(f"Failed to download PDF: {pdf_url}")
        return None

# Step 3: Extract text from PDFs
extracted_text_list = [""] * len(df)  # Initialize empty list for extracted text

for record_id in unique_record_ids:
    # Fetch all PDFs from the Zenodo record
    pdf_files = get_zenodo_pdfs(record_id)

    for pdf_number, pdf_file in enumerate(pdf_files):
        pdf_url = pdf_file["links"]["self"]
        pdf_filename = pdf_file["key"]
        local_pdf_path = os.path.join(pdf_folder, pdf_filename)

        print(f"Processing {pdf_filename} from Zenodo Record {record_id}")

        # Check if the file already exists locally
        if os.path.exists(local_pdf_path):
            pdf_bytes = open(local_pdf_path, "rb")
        else:
            pdf_bytes = download_pdf(pdf_url)
            if pdf_bytes:
                # Save PDF locally
                with open(local_pdf_path, "wb") as f:
                    f.write(pdf_bytes.getbuffer())

        if not pdf_bytes:
            continue

        # Open PDF and extract slides
        with pdfplumber.open(pdf_bytes) as pdf:
            slides = pdf.pages  # List of all slides
            extracted_text = [page.extract_text() if page.extract_text() else "" for page in slides]

            # Assign extracted text to the corresponding slides in df
            for idx, row in df.iterrows():
                if row["zenodo_filename"] == pdf_filename and row["page_number"] <= len(extracted_text):
                    extracted_text_list[idx] = extracted_text[row["page_number"] - 1]

# Step 4: Append extracted text to the dataframe
df["extracted_text"] = extracted_text_list

# Display the updated dataframe
df.head()


Processing 2023-Moore-N4BI-AHM-Welcome.pdf from Zenodo Record 10008464
Processing 2023-Moore-N4BI-AHM-Welcome.pdf from Zenodo Record 10008465
Processing Bio-Image_Data_Strudel_TU-Dresden_TP_Workshop_2023.pdf from Zenodo Record 10083555
Processing ECBIA_2024_EF.pdf from Zenodo Record 10679054
Processing SWAT4HCLS_2024_v11.pdf from Zenodo Record 10687658
Processing LLMs_BIA_v3.pdf from Zenodo Record 10815329
Processing Cultivating Open Training_v3.pdf from Zenodo Record 10816895
Processing FLIMfit_GerBi-FLIM-2024_Anca Margineanu.pdf from Zenodo Record 10886749
Processing FLUTE-software_GerBI-FLIM-2024_Chiara-Stringari.pdf from Zenodo Record 10886749
Processing napari-FLIM-phasor-plotter_GerBI-FLIM-2024_Conni-Wetzker+Marcelo-Zoccoler.pdf from Zenodo Record 10886749
Processing 20240410_NFDI4BIOIMAGE_RDM_Bio-Medicine_Schmidt.pdf from Zenodo Record 10939519
Processing Stefano_Della_Chiesa_2nd_SaxFDM-Beratungsstammtisch_v01.pdf from Zenodo Record 10942559
Processing Intro_DM_de.pdf from Zenod

,key,zenodo_record_id,zenodo_filename,page_number,text,visual,mixed,extracted_text
0,record10008464_pdf1_slide1,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,1,"[0.8054575, -0.4204579, 0.111390926, 0.2636367...","[0.22024887800216675, 0.6564452648162842, 0.02...","[0.22330284, -0.5643485, 0.32713842, -0.074667...","Welcome to\nBioImage Town!\nJosh Moore, Senior..."
1,record10008464_pdf1_slide2,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,2,"[1.1522328, 0.02467385, 0.23145455, 0.17258961...","[-0.3846272826194763, -0.01668522134423256, -0...","[0.34757608, -0.6562839, 0.5335755, -0.1906935...",Special welcome\nHonored New\nGuests Colleague...
2,record10008464_pdf1_slide3,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,3,"[1.0057557, 0.18344171, 0.03796136, 0.23549518...","[-0.32376205921173096, 0.06897055357694626, 0....","[0.06456853, -0.79703176, 0.871621, -0.8768049...",Value of bioimages\nhttps://ome-model.readthed...
3,record10008464_pdf1_slide4,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,4,"[1.2563236, 0.25261688, 0.0040982994, 0.202414...","[-0.03493745997548103, -0.11242333054542542, 0...","[0.3643795, -0.40593308, 0.97395766, -0.495041...",Value of bioimages\nAll-Hands Meeting · Josh M...
4,record10008464_pdf1_slide5,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,5,"[0.8471789, -0.40933934, 0.85754687, -0.496454...","[-0.6693567037582397, -0.02434205450117588, -0...","[-0.22939722, -0.6752343, 0.57570195, 0.046955...","Image Data Management isn’t always FAIR\n""Clar..."


In [8]:
df = df.rename(columns={
    "text": "text_embedding",
    "visual": "visual_embedding",
    "mixed": "mixed_embedding"
})

In [11]:
df.head()

,key,zenodo_record_id,zenodo_filename,page_number,text_embedding,visual_embedding,mixed_embedding,extracted_text
0,record10008464_pdf1_slide1,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,1,"[0.8054575, -0.4204579, 0.111390926, 0.2636367...","[0.22024887800216675, 0.6564452648162842, 0.02...","[0.22330284, -0.5643485, 0.32713842, -0.074667...","Welcome to\nBioImage Town!\nJosh Moore, Senior..."
1,record10008464_pdf1_slide2,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,2,"[1.1522328, 0.02467385, 0.23145455, 0.17258961...","[-0.3846272826194763, -0.01668522134423256, -0...","[0.34757608, -0.6562839, 0.5335755, -0.1906935...",Special welcome\nHonored New\nGuests Colleague...
2,record10008464_pdf1_slide3,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,3,"[1.0057557, 0.18344171, 0.03796136, 0.23549518...","[-0.32376205921173096, 0.06897055357694626, 0....","[0.06456853, -0.79703176, 0.871621, -0.8768049...",Value of bioimages\nhttps://ome-model.readthed...
3,record10008464_pdf1_slide4,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,4,"[1.2563236, 0.25261688, 0.0040982994, 0.202414...","[-0.03493745997548103, -0.11242333054542542, 0...","[0.3643795, -0.40593308, 0.97395766, -0.495041...",Value of bioimages\nAll-Hands Meeting · Josh M...
4,record10008464_pdf1_slide5,10008464,2023-Moore-N4BI-AHM-Welcome.pdf,5,"[0.8471789, -0.40933934, 0.85754687, -0.496454...","[-0.6693567037582397, -0.02434205450117588, -0...","[-0.22939722, -0.6752343, 0.57570195, 0.046955...","Image Data Management isn’t always FAIR\n""Clar..."


In [10]:
from datasets import Dataset
import pandas as pd

# Convert DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Remove the Pandas index column if it exists
hf_dataset = hf_dataset.remove_columns(["__index_level_0__"]) if "__index_level_0__" in hf_dataset.column_names else hf_dataset

hf_dataset.push_to_hub("ScaDS-AI/SlightInsight_Cache_w_Text")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ScaDS-AI/SlightInsight_Cache_w_Text/commit/003192dc27982060a48db63db6a04c8ada9a8fc2', commit_message='Upload dataset', commit_description='', oid='003192dc27982060a48db63db6a04c8ada9a8fc2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ScaDS-AI/SlightInsight_Cache_w_Text', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ScaDS-AI/SlightInsight_Cache_w_Text'), pr_revision=None, pr_num=None)